In [ ]:
import os
import json
import pandas as pd
import traceback

In [ ]:
# import os
# from huggingface_hub import InferenceClient
# from langchain.prompts import PromptTemplate

# # 1. Set your token (example with env var)
# # 2. Create the HF Inference Client
# # client = InferenceClient(
# #     model="gpt2", 
# #     token=os.environ["HF_TOKEN"]
# # )
# client = InferenceClient(
#     model="bigscience/bloomz-560m",
#     token=os.environ["HF_TOKEN"]
# )

# # 3. Define a LangChain-style prompt template
# prompt = PromptTemplate(
#     input_variables=["place"],
#     template="What is the capital of {place}?"
# )

# # 4. Format the prompt using LangChain
# final_prompt = prompt.format(place="France")

# # 5. Run text generation (not chat!)
# response = client.text_generation(
#     prompt=final_prompt,
#     max_new_tokens=50,
#     temperature=0.7,
# )

# # 6. Print the generated text
# print(response)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
load_dotenv()
# Your Hugging Face token
key=os.getenv("HF_TOKEN")

# Load model and tokenizer with token
model_id = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=key)
model = AutoModelForCausalLM.from_pretrained(model_id, use_auth_token=key)


In [ ]:

# Create pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=255
    do_sample=True,
    temperature=0.5,
    top_k=50,
    top_p=0.95
)
# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# Prompt template
prompt = PromptTemplate(
    input_variables=["name"],
    template="Can you tell me the current club joined by footballer {name}?"
)

# LangChain LLM chain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain
result = chain.run("Cole Palmer")
print(result)

In [ ]:
llm  # Model is working

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.llms import HuggingFacePipeline
from langchain.callbacks import get_openai_callback  # Optional: still usable for tracking, even without OpenAI

import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

load_dotenv()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
load_dotenv()
# Your Hugging Face token
key=os.getenv("HF_TOKEN")

# Load model and tokenizer with token
model_id = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_id, use_auth_token=hf_token)

# Create pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=255,
    do_sample=True,
    temperature=0.5,
    top_k=50,
    top_p=0.95
)
# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [ ]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {numbers} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check allthe questions to be conforming the text as well.
Make sure the format of your response is like RESPONSE_JSON below and use it as a guide.
Ensure to make {numbers} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
quiz_generator_prompt = PromptTemplate(
    input_variables=["text", "numbers","subject","tone","response_json"],
    template=TEMPLATE
)

In [ ]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generator_prompt,
    output_key="quiz",
    verbose=True
)

In [ ]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [ ]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "numbers", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
file_path=r"C:\Users\HP\mcqgen\data.txt"

In [ ]:
with open(file_path,'r') as f:
    data=f.read()

In [ ]:
print(data)

In [ ]:
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [ ]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": data,
            "numbers": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}") #Since using a local model so it won't be affected thereore count will be 0

In [ ]:
response

In [ ]:
quiz=response.get("quiz")
print(quiz)

In [ ]:
import re

# Extract JSON substring from quiz string using a simpler pattern
json_match = re.search(r'(\{.*\})', response["quiz"], re.DOTALL)
if json_match:
	quiz_json_str = json_match.group(1)
	try:
		quiz = json.loads(quiz_json_str)
	except json.JSONDecodeError as e:
		print("Extracted JSON string is malformed or incomplete.")
		print("Error:", e)
		print("Extracted string (truncated):", quiz_json_str[:500])
		quiz = None
else:
	raise ValueError("No valid JSON found in quiz string")

In [ ]:
quiz_table_data = []
if quiz is not None and isinstance(quiz, dict):
    for key, value in quiz.items():
        mcq = value["mcq"]
        options = " | ".join(
            [
                f"{option}: {option_value}"
                for option, option_value in value["options"].items()
            ]
        )
        correct = value["correct"]
        quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
else:
    print("Quiz data is not available or not in the expected format.")

In [ ]:
quiz_table_data

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("quiz.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')